In [15]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.linalg import cholesky

# Download 1-minute stock data for the last day (1-day period)
def download_stock_data(ticker, period='1mo', interval='5m'):
    stock_data = yf.download(ticker, period=period, interval=interval)
    return stock_data['Close']

# Fractional Brownian motion simulator
def fBM(T, n_steps, hurst_exponent=0.6):
    dt = T / n_steps
    t = np.linspace(0, T, n_steps + 1)
    
    R = np.zeros((n_steps + 1, n_steps + 1))
    for i in range(n_steps + 1):
        for j in range(n_steps + 1):
            R[i, j] = 0.5 * (abs(i*dt - j*dt)**(2*hurst_exponent) + abs(i*dt + j*dt)**(2*hurst_exponent) - 2*abs(i*dt)**(2*hurst_exponent)) # Corrected formula to ensure positive definiteness
    
    # Add a small perturbation to the diagonal to improve numerical stability
    R += np.eye(n_steps + 1) * 1e-8 
    
    L = cholesky(R, lower=True)
    W = np.random.normal(size=n_steps + 1)
    fBM = np.dot(L, W)
    
    return fBM

# Trading Strategy based on fBM
def fbm_trading_strategy(prices, fbm_path):
    initial_cash = 10000  # Starting with $100k
    position = 0  # Number of shares
    cash = initial_cash
    trades = []
    
    for i in range(1, len(prices)):
        if fbm_path[i] > fbm_path[i-1]:  # Uptrend: Buy
            if position == 0:
                shares_to_buy = cash // prices[i]
                cash -= shares_to_buy * prices[i]
                position += shares_to_buy
                trades.append(('Buy', prices.index[i], prices[i], position, cash))
        elif fbm_path[i] < fbm_path[i-1]:  # Downtrend: Sell
            if position > 0:
                cash += position * prices[i]
                trades.append(('Sell', prices.index[i], prices[i], position, cash))
                position = 0  # Liquidate position
    
    final_cash = cash + position * prices[-1]
    profit = final_cash - initial_cash
    return trades, profit

# Main function
def run_fbm_strategy(ticker):
    prices = download_stock_data(ticker)
    T = len(prices) - 1
    fbm_path = fBM(T, T, hurst_exponent=0.7)
    
    trades, profit = fbm_trading_strategy(prices, fbm_path)
    
    # print("Trade Log:")
    # for trade in trades:
    #     print(f"{trade[0]} at {trade[1]} - Price: {trade[2]:.2f}, Position: {trade[3]}, Cash: {trade[4]:.2f}")
    
    print(f"\nFinal Profit: ${profit:.2f}")

# Run the strategy for a specific ticker (e.g., AAPL)
run_fbm_strategy('TSLA')

[*********************100%***********************]  1 of 1 completed



Final Profit: $592.03


C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\928700533.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = cash // prices[i]
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\928700533.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cash -= shares_to_buy * prices[i]
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\928700533.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  trades.append(('Buy', prices.ind

In [12]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.linalg import cholesky

# Download 1-minute stock data for the last day (1-day period)
def download_stock_data(ticker, period='1mo', interval='1d'):
    stock_data = yf.download(ticker, period=period, interval=interval)
    return stock_data['Close']

# Fractional Brownian motion simulator
def fBM(T, n_steps, hurst_exponent=0.6):
    dt = T / n_steps
    t = np.linspace(0, T, n_steps + 1)
    
    R = np.zeros((n_steps + 1, n_steps + 1))
    for i in range(n_steps + 1):
        for j in range(n_steps + 1):
            R[i, j] = 0.5 * (abs(i*dt - j*dt)**(2*hurst_exponent) + abs(i*dt + j*dt)**(2*hurst_exponent) - 2*abs(i*dt)**(2*hurst_exponent)) # Corrected formula to ensure positive definiteness
    
    # Add a small perturbation to the diagonal to improve numerical stability
    R += np.eye(n_steps + 1) * 1e-8 
    
    L = cholesky(R, lower=True)
    W = np.random.normal(size=n_steps + 1)
    fBM = np.dot(L, W)
    
    return fBM

# Trading Strategy based on fBM
def fbm_trading_strategy(prices, fbm_path):
    initial_cash = 100000  # Starting with $100k
    position = 0  # Number of shares
    cash = initial_cash
    trades = []
    
    for i in range(1, len(prices)):
        if fbm_path[i] > fbm_path[i-1]:  # Uptrend: Buy
            if position == 0:
                shares_to_buy = cash // prices[i]
                cash -= shares_to_buy * prices[i]
                position += shares_to_buy
                trades.append(('Buy', prices.index[i], prices[i], position, cash))
        elif fbm_path[i] < fbm_path[i-1]:  # Downtrend: Sell
            if position > 0:
                cash += position * prices[i]
                trades.append(('Sell', prices.index[i], prices[i], position, cash))
                position = 0  # Liquidate position
    
    final_cash = cash + position * prices[-1]
    profit = final_cash - initial_cash
    return trades, profit


# Main function to loop over multiple stocks
def run_fbm_strategy_for_multiple_stocks(tickers):
    total_profit = 0
    results = []

    for ticker in tickers:
        print(f"Running strategy for {ticker}...")
        prices = download_stock_data(ticker)
        T = len(prices) - 1
        
        if T <= 0:
            print(f"Not enough data for {ticker}. Skipping...")
            continue
        
        fbm_path = fBM(T, T, hurst_exponent=0.7)
        trades, profit = fbm_trading_strategy(prices, fbm_path)
        
        total_profit += profit
        results.append({'Ticker': ticker, 'Profit': profit})
        
        print(f"Profit for {ticker}: ${profit:.2f}\n")
    
    print(f"\nCumulative Profit from all stocks: ${total_profit:.2f}")
    return results, total_profit

# Top 10 stocks (you can adjust this list based on market preference)
top_10_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'FB', 'JPM', 'V', 'UNH']

# Run the strategy for the top 10 stocks and calculate cumulative profit
results, total_profit = run_fbm_strategy_for_multiple_stocks(top_10_stocks)

# Display results
results_df = pd.DataFrame(results)
print("\nResults for each stock:\n", results_df)


[*********************100%***********************]  1 of 1 completed

Running strategy for AAPL...
Profit for AAPL: $-939.51

Running strategy for MSFT...



C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = cash // prices[i]
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cash -= shares_to_buy * prices[i]
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  trades.append(('Buy', prices.in

Profit for MSFT: $660.01

Running strategy for GOOGL...
Profit for GOOGL: $2587.67

Running strategy for AMZN...


[*********************100%***********************]  1 of 1 completed
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = cash // prices[i]
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cash -= shares_to_buy * prices[i]
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a valu

...

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['FB']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")')
[*********************100%***********************]  1 of 1 completed
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = cash // prices[i]
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cash -= shares_to_buy * prices[i]
C:\Users\Acer\AppData\

Not enough data for FB. Skipping...
Running strategy for JPM...
Profit for JPM: $-1359.45

Running strategy for V...


[*********************100%***********************]  1 of 1 completed
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = cash // prices[i]
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cash -= shares_to_buy * prices[i]
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a valu

Profit for V: $3022.20

Running strategy for UNH...
Profit for UNH: $-2974.41


Cumulative Profit from all stocks: $14036.74

Results for each stock:
   Ticker        Profit
0   AAPL   -939.509750
1   MSFT    660.007996
2  GOOGL   2587.668152
3   AMZN   9338.872513
4   TSLA  12943.255508
5   NVDA  -9241.895401
6    JPM  -1359.450455
7      V   3022.201904
8    UNH  -2974.412964



C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = cash // prices[i]
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cash -= shares_to_buy * prices[i]
C:\Users\Acer\AppData\Local\Temp\ipykernel_40860\962602897.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  trades.append(('Buy', prices.in